#  Colab에서 PySpark 실행하는 방법
- 참조 : https://medium.com/@TheITspace/running-pyspark-on-google-colab-2552435972b3

In [1]:
# Colab에서 PySpark 사용하는 방법 1
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=c89016cd77cdcb172b99d493df0bbbde3092c718eb1fd11e657c38c9dd7baf33
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
# # Colab에서 PySpark 사용하는 방법 2
# !apt-get install openjdk-8-jdk-headless -qq 
# !wget -q !wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz  
# !tar -xf spark-3.3.2-bin-hadoop3.tgz
# !pip install -q findspark

# import os
# import findspark

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# findspark.init()
# findspark.find()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("demo")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

## ngrok를 사용하여 Spark UI 접속

In [5]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

--2023-04-24 05:34:33--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8379857 (8.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   7.99M  41.1MB/s    in 0.2s    

2023-04-24 05:34:34 (41.1 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8379857/8379857]



In [6]:
!tar xf ngrok-v3-stable-linux-amd64.tgz

In [7]:
#  !./ngrok authtoken <authtoken > # https://dashboard.ngrok.com/
!./ngrok authtoken 68EgMVst4xkntqLofArnK_7VuGPawLqhayeXRFfNLtH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
get_ipython().system_raw('./ngrok http 4050 &')

In [9]:
!ps -ef | grep ngrok

root        1206       1  0 05:34 ?        00:00:00 ./ngrok http 4050
root        1212     715  0 05:34 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root        1220    1212  0 05:34 ?        00:00:00 grep ngrok


In [10]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://b4ee-34-73-67-161.ngrok-free.app


## 데이터 로딩
- 데이터 출처 : https://www.kaggle.com/sdolezel/black-friday

In [12]:
df = spark.read.csv("data.csv", header=True, inferSchema=True)

In [14]:
df.printSchema()

root
 |-- date_time: long (nullable = true)
 |-- b_code: string (nullable = true)
 |-- usage: double (nullable = true)
 |-- peak: double (nullable = true)



In [15]:
df.show(5)

+------------+------+------+------+
|   date_time|b_code| usage|  peak|
+------------+------+------+------+
|202201010000|  SGBD|237.06|948.24|
|202201010015|  SGBD|236.88|947.52|
|202201010030|  SGBD|236.88|947.52|
|202201010045|  SGBD| 234.9| 939.6|
|202201010100|  SGBD|234.54|938.16|
+------------+------+------+------+
only showing top 5 rows



In [16]:
df.count()

1000

## PySpark 사용법

### 특정 컬럼 데이터 보기

In [19]:
df.select("date_time","b_code","usage").show(5)

+------------+------+------+
|   date_time|b_code| usage|
+------------+------+------+
|202201010000|  SGBD|237.06|
|202201010015|  SGBD|236.88|
|202201010030|  SGBD|236.88|
|202201010045|  SGBD| 234.9|
|202201010100|  SGBD|234.54|
+------------+------+------+
only showing top 5 rows



### 컬럼

In [20]:
df.describe().show()

+-------+-----------------+------+------------------+------------------+
|summary|        date_time|b_code|             usage|              peak|
+-------+-----------------+------+------------------+------------------+
|  count|             1000|  1000|              1000|              1000|
|   mean|2.022010583445E11|  null|299.52450000000005|1198.0980000000002|
| stddev|30109.66929658576|  null| 59.51741340690685| 238.0696536276274|
|    min|     202201010000|  SGBD|             225.9|             903.6|
|    max|     202201110945|  SGBD|            431.28|           1725.12|
+-------+-----------------+------+------------------+------------------+



### 범주형 컬럼(Categorical columns)의 유일값 확인

In [22]:
df.select("b_code").distinct().show()

+------+
|b_code|
+------+
|  SGBD|
+------+



### Groupby 집계


In [23]:
from pyspark.sql import functions as F
df.groupBy("b_code").agg(F.sum("usage")).show()

+------+------------------+
|b_code|        sum(usage)|
+------+------------------+
|  SGBD|299524.50000000006|
+------+------------------+



## 데이터 저장

In [26]:
df.write.csv("pre_processed_data")

In [27]:
df.rdd.getNumPartitions()

1

## Pandas 데이터프레임으로 저장

In [28]:
df_pd = df.toPandas()
df_pd.to_csv("pandas_pre_processed_data.csv")